In [1]:
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

In [2]:
num_classes = 4
image_size = 224
batch_size_training = 10
batch_size_validation = 10

In [3]:
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input,)

In [4]:
train_generator = data_generator.flow_from_directory(
    'New folder/train',
    target_size = (image_size, image_size),
    batch_size = batch_size_training,
    class_mode = 'categorical'
    )

validation_generator = data_generator.flow_from_directory(
    'New folder/validation',
    target_size = (image_size, image_size),
    batch_size = batch_size_training,
    class_mode = 'categorical'
    )

Found 400 images belonging to 4 classes.
Found 9 images belonging to 4 classes.


In [5]:
#defingin the model
model = Sequential()

In [27]:
#adding VGG16 model
from xml.etree.ElementInclude import include


model.add(
    VGG16(include_top = False, pooling='avg', weights ='imagenet', input_shape= 224)
    )

TypeError: object of type 'int' has no len()

In [9]:
model.layers

In [8]:
model.layers[0].layers

In [11]:
model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4, activation="softmax"))

In [12]:
model.layers

In [13]:
for layer in model.layers:

    print(layer, layer.trainable)

<keras.engine.functional.Functional object at 0x0000026BD164CD00> True
<keras.layers.reshaping.flatten.Flatten object at 0x0000026BD139C6A0> True
<keras.layers.core.dense.Dense object at 0x0000026BD1683D00> True
<keras.layers.core.dense.Dense object at 0x0000026BD167BAC0> True
<keras.layers.core.dense.Dense object at 0x0000026BC7412C10> True


In [14]:
#training the model
model.layers[0].trainable = False
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics =['accuracy'])
steps_per_epoch_train = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
number_epochs = 2

In [15]:
for layer in model.layers:

    print(layer, layer.trainable)

<keras.engine.functional.Functional object at 0x0000026BD164CD00> False
<keras.layers.reshaping.flatten.Flatten object at 0x0000026BD139C6A0> True
<keras.layers.core.dense.Dense object at 0x0000026BD1683D00> True
<keras.layers.core.dense.Dense object at 0x0000026BD167BAC0> True
<keras.layers.core.dense.Dense object at 0x0000026BC7412C10> True


In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 4096)              2101248   
                                                                 
 dense_1 (Dense)             (None, 4096)              16781312  
                                                                 
 dense_2 (Dense)             (None, 4)                 16388     
                                                                 
Total params: 33,613,636
Trainable params: 18,898,948
Non-trainable params: 14,714,688
_________________________________________________________________


In [19]:
fit_history = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch_train, epochs=number_epochs,verbose=1, validation_data=validation_generator, validation_steps=steps_per_epoch_validation,)

Epoch 1/2


C:\Users\ASUS\AppData\Local\Temp\ipykernel_20788\2980021967.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fit_history = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch_train, epochs=number_epochs,verbose=1, validation_data=validation_generator, validation_steps=steps_per_epoch_validation,)


40/40 [==============================] - 40s 978ms/step - loss: 2.7418 - accuracy: 0.8950 - val_loss: 0.0049 - val_accuracy: 1.0000
Epoch 2/2
40/40 [==============================] - 45s 1s/step - loss: 1.4590e-06 - accuracy: 1.0000 - val_loss: 2.8875e-06 - val_accuracy: 1.0000


In [20]:
model.save('model/VGG16_latter_training/vgg16_Product_Classification1.h5')

In [21]:
from keras.models import load_model
vgg16_saved = load_model('model/VGG16/vgg16_Product_Classification_2Dense.h5')

In [22]:
from keras.models import Model
basemodel = Model(inputs=vgg16_saved.input, outputs=vgg16_saved.get_layer('dense').output)

In [23]:
import numpy as np
from numpy.linalg import norm

In [24]:
#predicting
from tensorflow.keras.preprocessing.image import load_img
# load an image from file
image1 = load_img('New folder/test/test4.jpg', target_size=(224, 224))
image2 = load_img('New folder/test/test3.png', target_size=(224, 224))


from tensorflow.keras.preprocessing.image import img_to_array
image1 = img_to_array(image1)
image2 = img_to_array(image2)

# reshape data for the model
image1 = image1.reshape((1, image1.shape[0], image1.shape[1], image1.shape[2]))
image2 = image2.reshape((1, image2.shape[0], image2.shape[1], image2.shape[2]))


from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image1 = preprocess_input(image1)
image2 = preprocess_input(image2)

# predict the probability across all output classes
yhat1_base = basemodel.predict(image1)
yhat2_base = basemodel.predict(image2)

#cosine similarity
A = yhat1_base[0]
B = yhat2_base[0]

cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)

1/1 [==============================] - 0s 166ms/step
Cosine Similarity: 0.9403707


In [25]:
#predicting
from tensorflow.keras.preprocessing.image import load_img
# load an image from file
image1 = load_img('New folder/test/test4.jpg', target_size=(224, 224))
image2 = load_img('New folder/test/test.png', target_size=(224, 224))


from tensorflow.keras.preprocessing.image import img_to_array
image1 = img_to_array(image1)
image2 = img_to_array(image2)

# reshape data for the model
image1 = image1.reshape((1, image1.shape[0], image1.shape[1], image1.shape[2]))
image2 = image2.reshape((1, image2.shape[0], image2.shape[1], image2.shape[2]))


from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image1 = preprocess_input(image1)
image2 = preprocess_input(image2)

# predict the probability across all output classes
yhat1_base = basemodel.predict(image1)
yhat2_base = basemodel.predict(image2)

#cosine similarity
A = yhat1_base[0]
B = yhat2_base[0]

cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)

1/1 [==============================] - 0s 134ms/step
Cosine Similarity: 0.83852446


In [26]:
#predicting
from tensorflow.keras.preprocessing.image import load_img
# load an image from file
image1 = load_img('New folder/test/test.png', target_size=(224, 224))
image2 = load_img('New folder/test/test3.png', target_size=(224, 224))


from tensorflow.keras.preprocessing.image import img_to_array
image1 = img_to_array(image1)
image2 = img_to_array(image2)

# reshape data for the model
image1 = image1.reshape((1, image1.shape[0], image1.shape[1], image1.shape[2]))
image2 = image2.reshape((1, image2.shape[0], image2.shape[1], image2.shape[2]))


from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image1 = preprocess_input(image1)
image2 = preprocess_input(image2)

# predict the probability across all output classes
yhat1_base = basemodel.predict(image1)
yhat2_base = basemodel.predict(image2)

#cosine similarity
A = yhat1_base[0]
B = yhat2_base[0]

cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)

1/1 [==============================] - 0s 137ms/step
Cosine Similarity: 0.87827444
